In [ ]:
import numpy as np
import pandas as pd

from gtda.time_series import SlidingWindow
from gtda.diagrams import PersistenceLandscape
from gtda.homology import VietorisRipsPersistence

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
pd.set_option('display.max_columns', None)
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
# !pip install spectrum 
# from spectrum.periodogram import speriodogram

import statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess

from pylab import *

import matplotlib.gridspec as gridspec

In [ ]:
# Functions to compute the Lp norms
# Find the range of x values from the persistence diagram:
def Ftseq(diagram):
    births =[]
    deaths =[]
    for pair in diagram:
        if pair[2] == 1:
            births.append(pair[0])
            deaths.append(pair[1])
    return np.linspace(min(births), max(deaths), 100)

# Calculate Lp norm:
def Lpnorm(tseq, landscapevalues, p = 1):
    norms = []
    if p == 'aucoriginal':
        layervalues = landscapevalues[layers]
        for point in zip(tseq,layervalues):
            norms.append(np.trapz(layervalues, tseq))
        return(sum(norms))
    elif p == 'aucUpdated':
        layervalues = landscapevalues[layers]
        return (np.trapz(layervalues))
    else:
        normvalues = []
        for layer in range(layers, 2*layers):
            layervalues = landscapevalues[layer]
            normvalue = np.linalg.norm(layervalues,p)**p
            if normvalue == 0:
                break
            else: 
                normvalues.append(normvalue)
        return (np.sum(normvalues)**(1/p))

In [ ]:
x = np.arange(0, 80, .1)
f1 =  math.pi * (1/4)
y1 = (np.sin(f1*x) 
#             + np.random.normal(scale=.1, size=len(x))

        )
# y1 = np.round(y1, 5)
plt.figure(figsize = (20,8))
plt.plot(x,y1)
plt.show()
print('period = ' + str(2*math.pi/f1))

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()
p.plot(sides='onesided')
plt.title('Periodogram of Time Series 1')
# plt.xlim([0,.1])

In [ ]:
df = pd.DataFrame({'x':x,
              'y':y1})
pd.set_option('display.max_rows', None)
df.head(100)

In [ ]:
simulated_data = pd.DataFrame({'Simulated Data 1' : y1
             })
simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(0)
simulated_data = simulated_data.dropna()
(simulated_data['Simulated Data 1'] == simulated_data['Simulated Data 2']).value_counts()

In [ ]:
simulated_data = pd.DataFrame({'Simulated Data 1' : y1
             })
simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(80)
simulated_data = simulated_data.dropna()
(simulated_data['Simulated Data 1'] == simulated_data['Simulated Data 2']).value_counts()

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()

for lag in range(10,20, 10):
    simulated_data = pd.DataFrame({'Simulated Data 1' : y1
                 })
    simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(lag)
    simulated_data = simulated_data.dropna()

    # Create point cloud
    window_size = 160
    stride = 1
    df = simulated_data
    X = df
    y = df.index
    SW = SlidingWindow(size=window_size, stride=stride)
    X_sw, yr = SW.fit_transform_resample(X, y)

    # Calculate the norms for each of the windows in the multivariate sliding window
    variabilitys = np.empty(window_size -1)
    variabilitys[:] = np.nan
    variabilitys = list(variabilitys)

    correlations = np.empty(window_size -1)
    correlations[:] = np.nan
    correlations = list(correlations)
    
    Norms = np.empty(window_size-1)
    Norms[:] = np.nan
    Norms = list(Norms)
    
    vrp = VietorisRipsPersistence()
    pl = PersistenceLandscape()
    for pointcloud in X_sw:
        persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
        landscapedata = pl.fit_transform(persistencediagram)
        tseq = Ftseq(persistencediagram[0])
        yvalues = landscapedata[0][1]
        Norms.append(Lpnorm(tseq, yvalues, p = 'auc'))    
    
        pointcloudcovmatrix = np.array(pd.DataFrame(pointcloud).cov())
        variabilitys.append((pointcloudcovmatrix[0][0]  + pointcloudcovmatrix[1][1]) / 2)
    
        pointcloudcorrmatrix = np.array(pd.DataFrame(pointcloud).corr())
        correlations.append(pointcloudcorrmatrix[0][1])
    

    
    min_max_scaler = MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(simulated_data.values)
    data_scaled = pd.DataFrame(data_scaled, columns = simulated_data.columns, index = simulated_data.index)
#     data_scaled['L1 Norm'] = min_max_scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)
#     data_scaled['Variability'] = min_max_scaler.fit_transform(np.array(variabilitys).reshape(-1, 1)).reshape(-1)
    data_scaled['L1 Norm'] = np.array(Norms) / pd.Series(Norms).max() if pd.Series(Norms).max() != 0 else np.array(Norms)*0
    data_scaled['Variability'] = np.array(variabilitys) / pd.Series(variabilitys).max() if pd.Series(variabilitys).max() != 0 else np.array(variabilitys)*0
    data_scaled['Correlation'] = min_max_scaler.fit_transform(np.array(correlations).reshape(-1, 1)).reshape(-1)
    
    
#     plt.figure(figsize = (20,12), facecolor = 'white')
#     plt.subplot(2,1,1)
#     plt.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
#     plt.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
#     plt.title('Lag ' + str(lag))
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(2,1,2)
#     plt.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
#     plt.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(3,1,3)
#     plt.plot(data_scaled['Variability'], color = 'purple', label = 'Variability')
#     plt.plot(data_scaled['Correlation'], color = 'orange', label = 'Correlation')
#     plt.xlim([0, 1000])
#     plt.legend()
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - No Noise' + ' ' +str(lag))
#     plt.show()
    
    
    Gtop = gridspec.GridSpec(3, 3, hspace = 0)
    Gbot = gridspec.GridSpec(3, 3, hspace = 1, wspace = .5)
    
    plt.figure(figsize=(12, 8))
    

    ax1 = plt.subplot(Gtop[0,:])
    ax1.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
    ax1.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
    plt.title('Lag ' + str(lag), fontsize=10)
    plt.legend()
    ax2 = plt.subplot(Gtop[1,:], sharex = ax1, sharey = ax1)
    ax2.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
    ax2.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
    ax2.plot(correlations, color = 'orange', label = 'Correlation')
    plt.legend()

    plt.subplot(Gbot[2,0])
    p.plot(sides='onesided')
    plt.title('Periodogram of Time Series 1')
    plt.subplot(Gbot[2, 1])
    plt.csd(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('CSD of TS1 vs TS2')
    plt.subplot(Gbot[2, 2])
    sns.scatterplot(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('Simulated Data Scatter Plot')
    
#     plt.suptitle('Lag ' + str(lag), fontsize=10)
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - No Noise' + ' ' +str(lag))
    plt.show()
    
    

In [ ]:
plt.plot(y1)

## Moderate Noise

In [ ]:
np.random.seed(42)
x = np.linspace(0, 100, 1000)
f1 =  math.pi * (1/4)
y1 = (np.sin(f1*x) 
            + np.random.normal(scale=.3, size=len(x))

        )
plt.figure(figsize = (20,8))
plt.plot(x,y1)
plt.show()
print('period = ' + str(2*math.pi/f1))

In [ ]:
# go through window by window
# see whats going on in the 0 noise, why l1's are rising and rectangular
# show rips diagram, scatter plot by windows
# add correlation plots

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()

for lag in range(0,90, 10):
    simulated_data = pd.DataFrame({'Simulated Data 1' : y1
                 })
    simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(lag)
    simulated_data = simulated_data.dropna()

    # Create point cloud
    window_size = 150
    stride = 1
    df = simulated_data
    X = df
    y = df.index
    SW = SlidingWindow(size=window_size, stride=stride)
    X_sw, yr = SW.fit_transform_resample(X, y)

    # Calculate the norms for each of the windows in the multivariate sliding window
    variabilitys = np.empty(window_size -1)
    variabilitys[:] = np.nan
    variabilitys = list(variabilitys)

    correlations = np.empty(window_size -1)
    correlations[:] = np.nan
    correlations = list(correlations)
    
    Norms = np.empty(window_size-1)
    Norms[:] = np.nan
    Norms = list(Norms)
    layers = 100
    vrp = VietorisRipsPersistence()
    pl = PersistenceLandscape(layers)
    for pointcloud in X_sw:
        persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
        landscapedata = pl.fit_transform(persistencediagram)
        tseq = Ftseq(persistencediagram[0])
        Norms.append(Lpnorm(tseq, landscapedata[0], p = 'aucUpdated'))    
    
        pointcloudcovmatrix = np.array(pd.DataFrame(pointcloud).cov())
        variabilitys.append((pointcloudcovmatrix[0][0]  + pointcloudcovmatrix[1][1]) / 2)
    
        pointcloudcorrmatrix = np.array(pd.DataFrame(pointcloud).corr())
        correlations.append(pointcloudcorrmatrix[0][1])
    

    
    min_max_scaler = MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(simulated_data.values)
    data_scaled = pd.DataFrame(data_scaled, columns = simulated_data.columns, index = simulated_data.index)
    data_scaled['L1 Norm'] = min_max_scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)
    data_scaled['Variability'] = min_max_scaler.fit_transform(np.array(variabilitys).reshape(-1, 1)).reshape(-1)
    data_scaled['Correlation'] = min_max_scaler.fit_transform(np.array(correlations).reshape(-1, 1)).reshape(-1)
    
    
#     plt.figure(figsize = (20,12), facecolor = 'white')
#     plt.subplot(2,1,1)
#     plt.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
#     plt.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
#     plt.title('Lag ' + str(lag))
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(2,1,2)
#     plt.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
#     plt.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(3,1,3)
#     plt.plot(data_scaled['Variability'], color = 'purple', label = 'Variability')
#     plt.plot(data_scaled['Correlation'], color = 'orange', label = 'Correlation')
#     plt.xlim([0, 1000])
#     plt.legend()
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - Moderate Noise' + ' ' +str(lag))
#     plt.show()
    
    Gtop = gridspec.GridSpec(3, 3, hspace = 0)
    Gbot = gridspec.GridSpec(3, 3, hspace = 1, wspace = .5)
    
    plt.figure(figsize=(12, 8))
    

    ax1 = plt.subplot(Gtop[0,:])
    ax1.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
    ax1.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
    plt.title('Lag ' + str(lag), fontsize=10)
    plt.legend()
    ax2 = plt.subplot(Gtop[1,:], sharex = ax1)
    ax2.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
    ax2.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
    plt.legend()

    plt.subplot(Gbot[2,0])
    p.plot(sides='onesided')
    plt.title('Periodogram of Time Series 1')
    plt.subplot(Gbot[2, 1])
    plt.csd(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('CSD of TS1 vs TS2')
    plt.subplot(Gbot[2, 2])
    sns.scatterplot(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('Simulated Data Scatter Plot')
    
#     plt.suptitle('Lag ' + str(lag), fontsize=10)
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - Moderate Noise' + ' ' +str(lag))
    plt.show()

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()

for lag in range(0,90, 10):
    simulated_data = pd.DataFrame({'Simulated Data 1' : y1
                 })
    simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(lag)
    simulated_data = simulated_data.dropna()

    # Create point cloud
    window_size = 150
    stride = 1
    df = simulated_data
    X = df
    y = df.index
    SW = SlidingWindow(size=window_size, stride=stride)
    X_sw, yr = SW.fit_transform_resample(X, y)

    # Calculate the norms for each of the windows in the multivariate sliding window
    variabilitys = np.empty(window_size -1)
    variabilitys[:] = np.nan
    variabilitys = list(variabilitys)

    correlations = np.empty(window_size -1)
    correlations[:] = np.nan
    correlations = list(correlations)
    
    Norms = np.empty(window_size-1)
    Norms[:] = np.nan
    Norms = list(Norms)
    layers = 100
    vrp = VietorisRipsPersistence()
    pl = PersistenceLandscape(layers)
    for pointcloud in X_sw:
        persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
        landscapedata = pl.fit_transform(persistencediagram)
        tseq = Ftseq(persistencediagram[0])
        Norms.append(Lpnorm(tseq, landscapedata[0], p = 1))    
    
        pointcloudcovmatrix = np.array(pd.DataFrame(pointcloud).cov())
        variabilitys.append((pointcloudcovmatrix[0][0]  + pointcloudcovmatrix[1][1]) / 2)
    
        pointcloudcorrmatrix = np.array(pd.DataFrame(pointcloud).corr())
        correlations.append(pointcloudcorrmatrix[0][1])
    

    
    min_max_scaler = MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(simulated_data.values)
    data_scaled = pd.DataFrame(data_scaled, columns = simulated_data.columns, index = simulated_data.index)
    data_scaled['L1 Norm'] = min_max_scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)
    data_scaled['Variability'] = min_max_scaler.fit_transform(np.array(variabilitys).reshape(-1, 1)).reshape(-1)
    data_scaled['Correlation'] = min_max_scaler.fit_transform(np.array(correlations).reshape(-1, 1)).reshape(-1)
    
    
#     plt.figure(figsize = (20,12), facecolor = 'white')
#     plt.subplot(2,1,1)
#     plt.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
#     plt.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
#     plt.title('Lag ' + str(lag))
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(2,1,2)
#     plt.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
#     plt.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
#     plt.xlim([0, 1000])
#     plt.legend()
    
#     plt.subplot(3,1,3)
#     plt.plot(data_scaled['Variability'], color = 'purple', label = 'Variability')
#     plt.plot(data_scaled['Correlation'], color = 'orange', label = 'Correlation')
#     plt.xlim([0, 1000])
#     plt.legend()
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - Moderate Noise' + ' ' +str(lag))
#     plt.show()
    
    Gtop = gridspec.GridSpec(3, 3, hspace = 0)
    Gbot = gridspec.GridSpec(3, 3, hspace = 1, wspace = .5)
    
    plt.figure(figsize=(12, 8))
    

    ax1 = plt.subplot(Gtop[0,:])
    ax1.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
    ax1.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
    plt.title('Lag ' + str(lag), fontsize=10)
    plt.legend()
    ax2 = plt.subplot(Gtop[1,:], sharex = ax1)
    ax2.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
    ax2.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
    plt.legend()

    plt.subplot(Gbot[2,0])
    p.plot(sides='onesided')
    plt.title('Periodogram of Time Series 1')
    plt.subplot(Gbot[2, 1])
    plt.csd(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('CSD of TS1 vs TS2')
    plt.subplot(Gbot[2, 2])
    sns.scatterplot(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('Simulated Data Scatter Plot')
    
#     plt.suptitle('Lag ' + str(lag), fontsize=10)
#     plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - Moderate Noise' + ' ' +str(lag))
    plt.show()

## Large Noise

In [ ]:
x = np.linspace(0, 100, 1000)
f1 =  math.pi * (1/4)
y1 = (np.sin(f1*x) 
            + np.random.normal(scale=1, size=len(x))

        )
plt.figure(figsize = (20,8))
plt.plot(x,y1)
plt.show()
print('period = ' + str(2*math.pi/f1))

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()


for lag in range(0,90, 10):
    simulated_data = pd.DataFrame({'Simulated Data 1' : y1
                 })
    simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(lag)
    simulated_data = simulated_data.dropna()

    # Create point cloud
    window_size = 150
    stride = 1
    df = simulated_data
    X = df
    y = df.index
    SW = SlidingWindow(size=window_size, stride=stride)
    X_sw, yr = SW.fit_transform_resample(X, y)

    # Calculate the norms for each of the windows in the multivariate sliding window
    variabilitys = np.empty(window_size -1)
    variabilitys[:] = np.nan
    variabilitys = list(variabilitys)

    correlations = np.empty(window_size -1)
    correlations[:] = np.nan
    correlations = list(correlations)
    
    Norms = np.empty(window_size-1)
    Norms[:] = np.nan
    Norms = list(Norms)
    vrp = VietorisRipsPersistence()
    pl = PersistenceLandscape()
    for pointcloud in X_sw:
        persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
        landscapedata = pl.fit_transform(persistencediagram)
        tseq = Ftseq(persistencediagram[0])
        yvalues = landscapedata[0][1]
        Norms.append(Lpnorm(tseq, yvalues, p = 'auc'))    
    
        pointcloudcovmatrix = np.array(pd.DataFrame(pointcloud).cov())
        variabilitys.append((pointcloudcovmatrix[0][0]  + pointcloudcovmatrix[1][1]) / 2)
    
        pointcloudcorrmatrix = np.array(pd.DataFrame(pointcloud).corr())
        correlations.append(pointcloudcorrmatrix[0][1])
    

    
    min_max_scaler = MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(simulated_data.values)
    data_scaled = pd.DataFrame(data_scaled, columns = simulated_data.columns, index = simulated_data.index)
    data_scaled['L1 Norm'] = min_max_scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)
    data_scaled['Variability'] = min_max_scaler.fit_transform(np.array(variabilitys).reshape(-1, 1)).reshape(-1)
    data_scaled['Correlation'] = min_max_scaler.fit_transform(np.array(correlations).reshape(-1, 1)).reshape(-1)
    

    Gtop = gridspec.GridSpec(3, 3, hspace = 0)
    Gbot = gridspec.GridSpec(3, 3, hspace = 1, wspace = .5)
    
    plt.figure(figsize=(12, 8))
    

    ax1 = plt.subplot(Gtop[0,:])
    ax1.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
    ax1.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
    plt.title('Lag ' + str(lag), fontsize=10)
    plt.legend()
    ax2 = plt.subplot(Gtop[1,:], sharex = ax1)
    ax2.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
    ax2.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
    plt.legend()

    plt.subplot(Gbot[2,0])
    p.plot(sides='onesided')
    plt.title('Periodogram of Time Series 1')
    plt.subplot(Gbot[2, 1])
    plt.csd(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('CSD of TS1 vs TS2')
    plt.subplot(Gbot[2, 2])
    sns.scatterplot(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('Simulated Data Scatter Plot')
    
#     plt.suptitle('Lag ' + str(lag), fontsize=10)
    plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - Large Noise' + ' ' +str(lag))
    plt.show()    


## XLarge Noise

In [ ]:
x = np.linspace(0, 100, 1000)
f1 =  math.pi * (1/4)
y1 = (np.sin(f1*x) 
            + np.random.normal(scale=3, size=len(x))

        )
plt.figure(figsize = (20,8))
plt.plot(x,y1)
plt.show()
print('period = ' + str(2*math.pi/f1))

In [ ]:
from spectrum import Periodogram
p = Periodogram(y1
#                 , sampling=len(simulated_data1)
               )
p.run()

for lag in range(0,90, 10):
    simulated_data = pd.DataFrame({'Simulated Data 1' : y1
                 })
    simulated_data['Simulated Data 2'] = simulated_data['Simulated Data 1'].shift(lag)
    simulated_data = simulated_data.dropna()

    # Create point cloud
    window_size = 150
    stride = 1
    df = simulated_data
    X = df
    y = df.index
    SW = SlidingWindow(size=window_size, stride=stride)
    X_sw, yr = SW.fit_transform_resample(X, y)

    # Calculate the norms for each of the windows in the multivariate sliding window
    variabilitys = np.empty(window_size -1)
    variabilitys[:] = np.nan
    variabilitys = list(variabilitys)

    correlations = np.empty(window_size -1)
    correlations[:] = np.nan
    correlations = list(correlations)
    
    Norms = np.empty(window_size-1)
    Norms[:] = np.nan
    Norms = list(Norms)
    vrp = VietorisRipsPersistence()
    pl = PersistenceLandscape()
    for pointcloud in X_sw:
        persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
        landscapedata = pl.fit_transform(persistencediagram)
        tseq = Ftseq(persistencediagram[0])
        yvalues = landscapedata[0][1]
        Norms.append(Lpnorm(tseq, yvalues, p = 'auc'))    
    
        pointcloudcovmatrix = np.array(pd.DataFrame(pointcloud).cov())
        variabilitys.append((pointcloudcovmatrix[0][0]  + pointcloudcovmatrix[1][1]) / 2)
    
        pointcloudcorrmatrix = np.array(pd.DataFrame(pointcloud).corr())
        correlations.append(pointcloudcorrmatrix[0][1])
    

    
    min_max_scaler = MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(simulated_data.values)
    data_scaled = pd.DataFrame(data_scaled, columns = simulated_data.columns, index = simulated_data.index)
    data_scaled['L1 Norm'] = min_max_scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)
    data_scaled['Variability'] = min_max_scaler.fit_transform(np.array(variabilitys).reshape(-1, 1)).reshape(-1)
    data_scaled['Correlation'] = min_max_scaler.fit_transform(np.array(correlations).reshape(-1, 1)).reshape(-1)
    
    
    Gtop = gridspec.GridSpec(3, 3, hspace = 0)
    Gbot = gridspec.GridSpec(3, 3, hspace = 1, wspace = .5)
    
    plt.figure(figsize=(12, 8))
    

    ax1 = plt.subplot(Gtop[0,:])
    ax1.plot(data_scaled['Simulated Data 1'], color = 'blue', label = 'Simulated Data 1')
    ax1.plot(data_scaled['Simulated Data 2'], color = 'red', label = 'Simulated Data 2')
    plt.title('Lag ' + str(lag), fontsize=10)
    plt.legend()
    ax2 = plt.subplot(Gtop[1,:], sharex = ax1)
    ax2.plot(data_scaled['Variability'], color = 'blueviolet', label = 'Variability')
    ax2.plot(data_scaled['L1 Norm'], color = 'black', label = 'L1 Norm')
    plt.legend()

    plt.subplot(Gbot[2,0])
    p.plot(sides='onesided')
    plt.title('Periodogram of Time Series 1')
    plt.subplot(Gbot[2, 1])
    plt.csd(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('CSD of TS1 vs TS2')
    plt.subplot(Gbot[2, 2])
    sns.scatterplot(simulated_data['Simulated Data 1'], simulated_data['Simulated Data 2'])
    plt.title('Simulated Data Scatter Plot')
    
#     plt.suptitle('Lag ' + str(lag), fontsize=10)
    plt.savefig('/Users/brianlivian/Desktop/Sine Lag Delays - XL Noise' + ' ' +str(lag))
    plt.show()  
    


In [ ]:
len(Norms)